# NeuralNetwork MLP classifier Training

## import Libary

In [5]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords
import Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# from Sastrawi import POSTagger
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import string
from imblearn.over_sampling import SMOTE
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ssetiawan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ssetiawan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ssetiawan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Prepare dataset

## Feature Extraction

In [2]:
# import preprocessed dataset 
df_ready = pd.read_csv("sentiment_analysis_dataset_filtered_tokens.csv")
df_ready['preprocessed_text'] = df_ready['preprocessed_text'].fillna('')

# feature extraction
# perpare preprocessed and tokenized text data
tokenized_texts = df_ready['preprocessed_text']

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the data to obtain TF-IDF features from text
tfidf_features = tfidf_vectorizer.fit_transform(tokenized_texts)

In [10]:
df_ready.head()

,Unnamed: 0,label,tokens,tokens_sent,filtered_tokens,preprocessed_text
0,0,positive,"['warung', 'ini', 'dimiliki', 'oleh', 'pengusa...",['warung ini dimiliki oleh pengusaha pabrik ta...,"['warung', 'dimiliki', 'pengusaha', 'pabrik', ...",warung dimiliki pengusaha pabrik puluhan terke...
1,1,neutral,"['mohon', 'ulama', 'lurus', 'dan', 'k212', 'mm...",['mohon ulama lurus dan k212 mmbri hujjah part...,"['mohon', 'ulama', 'lurus', 'k212', 'mmbri', '...",mohon ulama lurus k212 mmbri hujjah partai diw...
2,2,positive,"['lokasi', 'strategis', 'di', 'jalan', 'sumatr...",['lokasi strategis di jalan sumatra bandung te...,"['lokasi', 'strategis', 'jalan', 'sumatra', 'b...",lokasi strategis jalan sumatra bandung nya nya...
3,3,positive,"['betapa', 'bahagia', 'nya', 'diri', 'ini', 's...",['betapa bahagia nya diri ini saat unboxing pa...,"['betapa', 'bahagia', 'nya', 'unboxing', 'pake...",betapa bahagia nya unboxing paket barang nya b...
4,4,negative,"['aduh', 'jadi', 'mahasiswa', 'jangan', 'sombo...",['aduh jadi mahasiswa jangan sombong dong kasi...,"['aduh', 'mahasiswa', 'sombong', 'kasih', 'kar...",aduh mahasiswa sombong kasih kartu kuning bela...


## model splitting

In [4]:
# Split train and test
X_train, X_test, y_train, y_test = train_test_split(
   tfidf_features, df_ready['label']
   , test_size=0.2, random_state=42
   )

# tackle imbalance 
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [5]:
from sklearn.neural_network import MLPClassifier


In [11]:
# Create an MLP Classifier
mlp_classifier = MLPClassifier(max_iter=1000)

# Fit the model on the training data
mlp_classifier.fit(X_train_smote, y_train_smote)


MLPClassifier(max_iter=1000)

In [12]:
# Predict on the test data
predictions_mlp = mlp_classifier.predict(X_test)


In [8]:
# Calculate evaluation metrics
accuracy_mlp = accuracy_score(y_test, predictions_mlp)
precision_mlp = precision_score(y_test, predictions_mlp, average='weighted')
recall_mlp = recall_score(y_test, predictions_mlp, average='weighted')
f1_mlp = f1_score(y_test, predictions_mlp, average='weighted')

# Print evaluation metrics
print("Logistic Regression Results:")
print(f"Accuracy: {accuracy_mlp:.2f}")
print(f"Precision: {precision_mlp:.2f}")
print(f"Recall: {recall_mlp:.2f}")
print(f"F1-Score: {f1_mlp:.2f}")

Logistic Regression Results:
Accuracy: 0.79
Precision: 0.80
Recall: 0.79
F1-Score: 0.79


In [8]:

# # Save the tuned logistic regression model 
model_mlp_filename = 'mlp_model.pkl'
# with open(model_mlp_filename, 'wb') as model_file:
#     joblib.dump(mlp_classifier, model_mlp_filename)

# Load the tuned logistic regression model 
with open(model_mlp_filename, 'rb') as model_file:
    loaded_model_mlp = joblib.load(model_mlp_filename)


# ## save and load tfidf_vectorizer
# # Save the tfidf_vectorizer used for training
tfidf_vectorizer_filename = 'tfidf_vectorizer.pkl'
# with open(tfidf_vectorizer_filename, 'wb') as vectorizer_file:
#    joblib.dump(tfidf_vectorizer, vectorizer_file)

# Load the tfidf_vectorizer used for training
with open(tfidf_vectorizer_filename, 'rb') as vectorizer_file:
   loaded_tfidf_vectorizer = joblib.load(tfidf_vectorizer_filename)


In [9]:
# def text_processing(text):
#    processed_text = text_cleansing(text)
#    processed_text = nltk.word_tokenize(processed_text)
#    processed_text = filter_stopwords(processed_text)
#    processed_text = stemming(processed_text)
#    return processed_text


def predict_NN (text):
   tfidf_vector = loaded_tfidf_vectorizer.transform([text])  
   sentiment = loaded_model_mlp.predict(tfidf_vector)
   return sentiment 


In [19]:
text = "waahahah gimana nih"

print (f"texs:{text}")
print (f"sentiment:{predict_NN(text)}")

texs:waahahah gimana nih
sentiment:['negative']
